## Process output of simulations

In [8]:
import os
import glob
import gzip
import math
import random
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points
import lxml.etree as ET
import tqdm
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, Subset
import torch_geometric
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import LineGraph
import processing_io as pio

districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

# Parameters to adapt
districts_of_policy_implementation = [5, 6, 7]
is_for_1pm = True
plot_in_percentage = True

string_is_for_1pm = "pop_1pm" if is_for_1pm else "pop_1pct"
string_district_of_interest = "_".join([str(d) for d in districts_of_policy_implementation])

path = "../../../../data/" +  string_is_for_1pm + "_simulations/"
basecase_subdir = pio.get_subdirs(path + string_is_for_1pm + "_basecase/")
comparison_subdir = pio.get_subdirs(path + string_is_for_1pm + "_policy_in_zone_2")

result_path_basecase_mean = "results/gdf_" + string_is_for_1pm + "_basecase_mean.geojson"
result_path_comparison_mean = "results/gdf_" + string_is_for_1pm + "_policy_in_" + string_district_of_interest + ".geojson"
result_path_difference = "results/gdf_" + string_is_for_1pm + "_difference.geojson"

In [9]:
random_seed_2_df_basecase = pio.create_dic(subdir=basecase_subdir)
random_seed_2_df_comparison = pio.create_dic(subdir = comparison_subdir)

In [10]:
geodataframes_basecase = list(random_seed_2_df_basecase.values())
geodataframes_comparison = list(random_seed_2_df_comparison.values())

In [11]:
gdf_basecase_mean = pio.compute_average_or_median_geodataframe(geodataframes=geodataframes_basecase, column_name="vol_car", is_mean=True)
gdf_comparison_mean = pio.compute_average_or_median_geodataframe(geodataframes=geodataframes_comparison, column_name="vol_car", is_mean=True)

In [12]:
gdf_comparison_mean_extended = pio.extend_geodataframe(gdf_base = gdf_basecase_mean, gdf_to_extend=gdf_comparison_mean, column_to_extend='osm:way:highway', new_column_name='highway')
gdf_basecase_mean = gdf_basecase_mean.rename(columns={"osm:way:highway": "highway"})
gdf_basecase_without_unnecessary_columns = pio.remove_columns(gdf_with_correct_columns=gdf_comparison_mean_extended, gdf_to_be_adapted=gdf_basecase_mean)

In [13]:
gdf_basecase_difference = pio.compute_difference_geodataframe(gdf_to_substract_from=gdf_comparison_mean_extended, gdf_to_substract=gdf_basecase_without_unnecessary_columns, column_name= 'vol_car')

In [14]:
# Save the dataframes to files
gdf_basecase_mean.to_file(result_path_basecase_mean, driver='GeoJSON')
gdf_comparison_mean_extended.to_file(result_path_comparison_mean, driver='GeoJSON')
gdf_basecase_difference.to_file(result_path_difference, driver='GeoJSON')

In [15]:
gdf_comparison_mean_extended

,link,from_node,to_node,length,freespeed,capacity,lanes,modes,vol_car,storageCapacityUsedInQsim,geometry,highway
0,100315,24972409,24972408,16.181257,8.333333,480.0,1.0,"bus,car,car_passenger",6.2,NaN,"LINESTRING (651473.304 6861511.110, 651457.123...",residential
1,100316,5904976363,24983651,14.860209,8.333333,240.0,1.0,"bus,car,car_passenger,pt",6.2,NaN,"LINESTRING (651477.818 6861578.659, 651475.677...",tertiary
2,100317,24983651,5904976363,14.860209,8.333333,480.0,2.0,"bus,car,car_passenger,pt",1.4,NaN,"LINESTRING (651475.677 6861563.954, 651477.818...",tertiary
3,100321,664205947,24972376,22.264540,8.333333,960.0,2.0,"car,car_passenger",7.8,NaN,"LINESTRING (651565.078 6861531.470, 651559.387...",residential
4,100324,24972376,24972375,64.853276,8.333333,480.0,1.0,"bus,car,car_passenger",8.2,NaN,"LINESTRING (651559.387 6861509.945, 651502.714...",residential
...,...,...,...,...,...,...,...,...,...,...,...,...
31211,pt_IDFM:480531,pt_IDFM:480531,pt_IDFM:480531,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,subway",0.0,NaN,"LINESTRING (649659.000 6866255.005, 649659.000...",NaN
31212,pt_IDFM:480541,pt_IDFM:480541,pt_IDFM:480541,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,subway",0.0,NaN,"LINESTRING (649661.000 6866250.005, 649661.000...",NaN
31213,pt_IDFM:480541_pt_IDFM:480286,pt_IDFM:480541,pt_IDFM:480286,699.892849,12.000000,7999.2,1.0,"artificial,subway",0.0,0.777581,"LINESTRING (649661.000 6866250.005, 649784.000...",NaN
31214,pt_IDFM:480964,pt_IDFM:480964,pt_IDFM:480964,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,tram",0.0,NaN,"LINESTRING (646505.000 6860021.005, 646505.000...",NaN
